Feature Engineering

a=EMA >=5 RSI>=70
b=EMA >=35, >89
c=EMA >=89 ; sideway == 5,15,35,89,=>200
d= EMA = 89 ; down trend = <=89, <200
e= EMA <=85,5; crash

In [12]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from cassandra.cluster import Cluster
from datetime import datetime


In [13]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('data_stock')

eps pe pbv percentyield marketcap


ทำไมตั้งแบบนี้

EPS>0 = บริษัทมีกำไร (คุณภาพพื้นฐานขั้นต่ำ)

P/E 8–25 = กรองถูก/แพงเกินไป (ค่า default ทั่วไป—ปรับได้ตามอุตสาหกรรม)

P/BV 0.8–2.5 = ไม่ต่ำกว่าทุนมากเกิน (เสี่ยง) และไม่แพงเวอร์

Dividend Yield ≥ 3% (≤ 20%) = ให้ผลตอบแทนเงินสดพอเหมาะ (กัน outlier yield สูงผิดปกติ)

Quantile mode ใช้สัดส่วนภายในชุดข้อมูล → ดีสำหรับเปรียบเทียบ “สัมพัทธ์” ในตลาด/อุตสาหกรรมเดียวกัน

ถ้าต้องการ “น้ำหนัก” ไม่เท่ากัน (เช่นให้ EPS/PE หนักกว่า PBV/Yield) บอกได้ ผมปรับให้เป็น weighted score ได้เลย เช่น EPS/PE = 2 คะแนน, อีกสองตัว=1 คะแนน แล้วค่อยแปลงเป็น A–E ตามสัดส่วนคะแนนครับ

In [17]:
rows_fin = session.execute("""
    SELECT symbol,close, pe, pbv, dividendyield,marketcap,bvps FROM smartset_finan_data ALLOW FILTERING
""")
df_financial = pd.DataFrame(rows_fin, columns=["symbol","close","pe","pbv","dividendyield","marketcap","bvps"])
df_financial = df_financial.dropna()

In [18]:
import numpy as np
import pandas as pd

def grade_financial(
    df,
    *,
    eps_col="eps",
    pe_col="pe",
    pbv_col="pbv",
    yld_col="percentyield",   # หน่วย % เช่น 3.0 = 3%
    mcap_col="marketcap",
    mode="fixed",             # "fixed" หรือ "quantile"
    # ----------- เกณฑ์โหมด fixed -----------
    pe_range=(8, 25),         # P/E ดี อยู่ในช่วง 8–25
    pbv_range=(0.8, 2.5),     # P/BV ดี อยู่ในช่วง 0.8–2.5
    yld_min=3.0,              # Yield ดี ≥ 3%
    yld_max=20.0,             # กัน outlier (ถ้าเกินมากมักมีเหตุผิดปกติ)
    use_mcap=False,           # เปิดใช้ marketcap เป็นปัจจัยที่ 5
    mcap_min_pct=0.5,         # ใช้เมื่อ use_mcap=True: ใหญ่กว่ามัธยฐาน (50th pct)
    # ----------- เกณฑ์โหมด quantile -----------
    pe_q=0.75,                # P/E ดี ≤ P75 ของค่า P/E ที่เป็นบวก
    pbv_q=0.60,               # P/BV ดี ≤ P60
    yld_q=0.75                # Yield ดี ≥ P75
):
    df = df.copy()

    # สร้าง EPS ถ้ายังไม่มี (ต้องมี close และ pe > 0)
    if eps_col not in df.columns and {"close", pe_col}.issubset(df.columns):
        df[eps_col] = np.where((df[pe_col] > 0) & df["close"].notna(), df["close"] / df[pe_col], np.nan)

    # เงื่อนไขพื้นฐาน (True/False) — จัด dtype ให้รองรับ NaN
    s_eps = (df.get(eps_col) > 0)

    if mode == "fixed":
        s_pe  = (df.get(pe_col) >= pe_range[0]) & (df.get(pe_col) <= pe_range[1])
        s_pbv = (df.get(pbv_col) >= pbv_range[0]) & (df.get(pbv_col) <= pbv_range[1])
        s_yld = (df.get(yld_col) >= yld_min) & (df.get(yld_col) <= yld_max)
    elif mode == "quantile":
        pe_pos = df.loc[df.get(pe_col, pd.Series(index=df.index)) > 0, pe_col]
        pe_cut = pe_pos.quantile(pe_q) if len(pe_pos) else np.inf
        s_pe  = df.get(pe_col) <= pe_cut

        pbv_cut = df.get(pbv_col).quantile(pbv_q) if pbv_col in df else np.inf
        s_pbv = df.get(pbv_col) <= pbv_cut

        yld_cut = df.get(yld_col).quantile(yld_q) if yld_col in df else -np.inf
        s_yld = df.get(yld_col) >= yld_cut
    else:
        raise ValueError("mode ต้องเป็น 'fixed' หรือ 'quantile'")

    masks = [s_eps, s_pe, s_pbv, s_yld]

    # ตัวเลือก: รวม marketcap เป็นเกณฑ์ที่ 5 (เช่น ใหญ่มากกว่ามัธยฐาน)
    if use_mcap and (mcap_col in df):
        mcap_cut = df[mcap_col].quantile(mcap_min_pct)
        s_mcap = df[mcap_col] >= mcap_cut
        masks.append(s_mcap)

    # คิดคะแนน (NaN -> 0)
    score = sum(m.fillna(False).astype(int) for m in masks)
    max_score = len(masks)

    # map เป็นเกรดตาม “สัดส่วนคะแนน”
    # A: ≥90%, B: ≥70%, C: ≥50%, D: ≥25%, E: ต่ำกว่านั้น
    pct = score / max_score
    grade = np.where(pct >= 0.90, "A",
             np.where(pct >= 0.70, "B",
             np.where(pct >= 0.50, "C",
             np.where(pct >= 0.25, "D", "E"))))

    return pd.Series(grade, index=df.index), score

# ----------------- ใช้งานกับ df_financial -----------------
# โหมด fixed (ค่าเริ่มแนะนำ)
df_financial["financial_level"], df_financial["financial_score"] = grade_financial(
    df_financial,
    mode="fixed",
    pe_range=(8, 25),
    pbv_range=(0.8, 2.5),
    yld_min=3.0,
    yld_max=20.0,
    use_mcap=False   # เปลี่ยนเป็น True ถ้าอยากนับ marketcap ด้วย
)

# หรือ โหมด quantile (ยืดหยุ่นกับแต่ละกลุ่ม/ตลาด)
# df_financial["financial_level"], df_financial["financial_score"] = grade_financial(
#     df_financial,
#     mode="quantile",
#     use_mcap=False
# )


TypeError: unsupported operand type(s) for /: 'decimal.Decimal' and 'float'